#### A Deeper Look Inside RetinaNet
- What does its anchor boxes look like? 
- What is each helper function doing?
- How to Recover BBox from its output?

In [1]:
import os
import sys
path = os.path.join(os.getcwd(), '..')
sys.path.append(path)

from pdb import set_trace
import warnings 
warnings.filterwarnings("ignore")

from fastai2.vision.all import *

from src.data.dblock import build_dblock
from src.data.dls import build_dataloaders
from src.model.model import get_retinanet, split_param_groups
from src.metrics.loss import get_retinanet_loss
from src.metrics.mAP import mAP

%load_ext autoreload
%autoreload 2

In [2]:
data_path = '/userhome/34/h3509807/wheat-data'

In [3]:
dls = build_dataloaders(
    data_path, bs = 8, 
    resize_sz = 256, norm = False, 
    rand_seed = 144, test_mode = True
    )